In [1]:
%%bash
# Buat folder cache di workspace (biar tidak makan storage utama 20GB)
mkdir -p /workspace/hf_cache

# Set environment variables permanen untuk session ini
echo 'export TRANSFORMERS_CACHE=/workspace/hf_cache' >> ~/.bashrc
echo 'export HF_HOME=/workspace/hf_cache' >> ~/.bashrc
echo 'export HF_HUB_CACHE=/workspace/hf_cache' >> ~/.bashrc

# Terapkan langsung ke session aktif
export TRANSFORMERS_CACHE=/workspace/hf_cache
export HF_HOME=/workspace/hf_cache
export HF_HUB_CACHE=/workspace/hf_cache

# Bersihkan cache lama yang makan ruang
rm -rf ~/.cache/huggingface
rm -rf ~/.cache/pip
echo "✅ Cache diarahkan ke /workspace/hf_cache"

✅ Cache diarahkan ke /workspace/hf_cache


In [2]:
!pip install transformers==4.38.2 accelerate==0.27.2 bitsandbytes==0.42.0 pydantic==2.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 114.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 49.7 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 128.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 791.7/791.7 kB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [9]:
!pip install peft==0.10.0 datasets==2.19.1


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import torch

In [ ]:
model_name = "deepseek-coder-6.7b-instruct"
model_id = "deepseek-ai/{}".format(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_id)

bnb = BitsAndBytesConfig(
    load_in_8bit=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=bnb
)

model = prepare_model_for_kbit_training(model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
lora_config = LoraConfig(
    r=16,                 # rank kecil → hemat VRAM
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [24]:
model = get_peft_model(model, lora_config)

In [5]:
from prompt_generator import CodeReviewPromptGenerator
prompt_generator = CodeReviewPromptGenerator()

def format_prompt(data):
    prompt = prompt_generator.generate_style_review_prompt(
        added_code=data["added_code"],
        deleted_code=data["deleted_code"],
        full_function_code=data["full_function_code"],
        function_name=data["function_name"],
    )
    target = data["code_review_suggestion"]
    return prompt, target

In [6]:
def tokenize_example(example, max_length=1024):  # sequence lebih pendek → hemat memori
    prompt, target = format_prompt(example)
    full_text = prompt + target
    tokens = tokenizer(
        full_text,
        truncation=True,
        max_length=max_length,
        padding="max_length"
    )
    return tokens

In [7]:
import json

dataset = []
with open("merged_dataset.json", "r") as f:
    dataset = json.load(f)

tokenized_data = [tokenize_example(e) for e in dataset]

NameError: name 'tokenizer' is not defined

In [8]:
for i in dataset[0:10]:
    prompt, target = format_prompt(i)
    print("-----" * 10)
    print(prompt)  # print only first 500 characters of the prompt

--------------------------------------------------
### Instruction:
You are a code reviewer. Analyze this Python code change and respond EXACTLY in the format below.

Full function `handle_api_response`:
```python
def handle_api_response(api_response):
    if api_response is not None:
        if isinstance(api_response, dict):
            if "status_code" in api_response:
                status_code = api_response["status_code"]
                if status_code == 200:
                    response_data = api_response["data"]
                    # Validate response data
                    if isinstance(response_data, list):
                        for item in response_data:
                            if not isinstance(item, dict):
                                raise ValueError("Invalid response data")
                    elif not isinstance(response_data, dict):
                        raise ValueError("Invalid response data")
                    return response_data
                e

In [18]:
dataset_hf = Dataset.from_list([{k: v for k, v in t.items()} for t in tokenized_data])
train_size = int(0.8 * len(dataset_hf))
train_dataset = dataset_hf.select(range(train_size))
test_dataset = dataset_hf.select(range(train_size, len(dataset_hf)))

def add_labels(batch):
    batch["labels"] = batch["input_ids"]
    return batch

train_dataset = train_dataset.map(add_labels)
test_dataset = test_dataset.map(add_labels)

Map:   0%|          | 0/891 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./{}_style_review".format(model_name),
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="epoch",
    learning_rate=2e-4,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    logging_steps=10,
    fp16=True,
    push_to_hub=False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("./{}_style_review".format(model_name))

In [ ]:
# -------------------------------
# 9️⃣ Generate output contoh
# -------------------------------
sample = dataset[0]
prompt, _ = format_prompt(sample)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))